>SCIKIT-LEARN

> **Group 23**
>> **Team-mates**
>>> **Madhav Jivani - 202201285**

>>> **Tarun Kumar**

>>> **Rupesh Singh**

In [26]:
# for using dataframes
import pandas as pd
# for data transformation or encoding and metric calculations
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score
# for feature selection and models
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

In [2]:
data = pd.read_csv("loan_status_data.csv")
data.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


**SEPARTE THE NUMERIC AND CATEGORICAL DATA**

In [3]:
x_num = data[["id","person_age","person_income","person_emp_length","loan_amnt","loan_int_rate","loan_percent_income","cb_person_cred_hist_length"]]

x_cat = data[["id","person_home_ownership","loan_intent","loan_grade","cb_person_default_on_file","loan_status"]]
x_num.head()

,id,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length
0,0,37,35000,0.0,6000,11.49,0.17,14
1,1,22,56000,6.0,4000,13.35,0.07,2
2,2,29,28800,8.0,6000,8.90,0.21,10
3,3,30,70000,14.0,12000,11.11,0.17,5
4,4,22,60000,2.0,6000,6.92,0.10,3


**CAPPING THE OUTLIERS TO 10 AND 90 PERCENTILE W.R.T. TO THEIR VALUES**

In [5]:
import pandas as pd

def remove_outliers(df):
    # Copy the original DataFrame to avoid modifying it directly
    cleaned_x_num = df.copy()
    
    # Iterate through each column
    for column in cleaned_x_num.columns:
        q75 = cleaned_x_num[column].quantile(0.75)
        q25 = cleaned_x_num[column].quantile(0.25)
        
        # Interquartile Range
        iqr = q75 - q25  
        lb = q25 - 1.5 * iqr  # Lower bound
        ub = q75 + 1.5 * iqr  # Upper bound
        
        # Mask the outliers
        cleaned_x_num[column] = cleaned_x_num[column].mask(
            (cleaned_x_num[column] < lb) | (cleaned_x_num[column] > ub)
        )
    
    # drop rows with any NaN values (fully remove rows with outliers)
    cleaned_x_num.dropna(inplace=True)
    
    return cleaned_x_num

# Use the function
cleaned_x_num = remove_outliers(x_num)
cleaned_x_num.shape


(49882, 8)

**ENCODING THE CATEGORICAL DATA**

In [6]:
onehot_encoder = OneHotEncoder(sparse_output=False) 
label_encoder = LabelEncoder()

# One-Hot Encoding for nominal columns
nominal_cols = ['person_home_ownership', 'loan_intent', 'cb_person_default_on_file', 'loan_status']
onehot_encoded = onehot_encoder.fit_transform(x_cat[nominal_cols])

# Create DataFrame for One-Hot Encoded variables
onehot_df = pd.DataFrame(onehot_encoded, columns=onehot_encoder.get_feature_names_out(nominal_cols))

# Label Encoding for ordinal columns
ordinal_col = 'loan_grade'
label_encoded = label_encoder.fit_transform(x_cat[ordinal_col])

# Create DataFrame for Label Encoded variable
label_encoded_df = pd.DataFrame(label_encoded, columns=[ordinal_col])

# Combine One-Hot Encoded, Label Encoded DataFrames, and the original id column
final_encoded_df = pd.concat([x_cat[['id']], onehot_df, label_encoded_df], axis=1)

**MERGE NUMERICAL AND CATEGORICAL DATA**

In [9]:
df = pd.merge(final_encoded_df, cleaned_x_num, on='id', how='inner')
def determine_status(row):
    if row['loan_status_0'] == 1:
        return 0
    elif row['loan_status_1'] == 1:
        return 1
    else:
        return -1  # or any other default value you choose

# Apply the function to each row
df['status'] = df.apply(determine_status, axis=1)
df.drop(columns=['loan_status_0', 'loan_status_1'], inplace=True)
df.head()


,id,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,...,cb_person_default_on_file_Y,loan_grade,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,status
0,0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1,37.0,35000.0,0.0,6000.0,11.49,0.17,14.0,0
1,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,2,22.0,56000.0,6.0,4000.0,13.35,0.07,2.0,0
2,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0,29.0,28800.0,8.0,6000.0,8.90,0.21,10.0,0
3,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,30.0,70000.0,14.0,12000.0,11.11,0.17,5.0,0
4,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0,22.0,60000.0,2.0,6000.0,6.92,0.10,3.0,0


**FEATURE SELECTION**

In [11]:
# Select features (X) and target variable (Y)
X = df.drop(["id","status"],axis=1)

y = df['status']

>1.) Using SelectKBest


In [12]:
selector = SelectKBest(score_func=f_classif, k=5)

# Fit the selector to the data (X: features, y: target)
X_new = selector.fit_transform(X, y)

# Get the selected feature names
selected_features = X.columns[selector.get_support()]

# Display results
print("Selected features:", selected_features)

# Optionally, convert the selected features back into a DataFrame
X_selected = pd.DataFrame(X_new, columns=selected_features)

# Display the transformed data with selected features
selected_features

selector.scores_
selector.pvalues_

Selected features: Index(['person_home_ownership_RENT', 'loan_grade', 'person_income',
       'loan_int_rate', 'loan_percent_income'],
      dtype='object')


array([0.00000000e+000, 2.68606367e-001, 2.20880775e-076, 0.00000000e+000,
       5.08809741e-037, 3.55670341e-035, 1.86390418e-024, 5.77141472e-022,
       4.38830886e-002, 7.57968145e-051, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 2.86913956e-001, 0.00000000e+000, 2.90274694e-129,
       1.66958963e-110, 0.00000000e+000, 0.00000000e+000, 7.69430123e-002])

>2.) Using RFE

In [13]:
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Standardize features
scaler = StandardScaler()
model = LogisticRegression(max_iter=500, solver='lbfgs', random_state=42)  # Increase max_iter

# Create a pipeline that scales data and applies RFE
pipeline = Pipeline([
    ('scaler', scaler),
    ('rfe', RFE(estimator=model, n_features_to_select=5))
])

# Fit RFE with scaled data
pipeline.fit(X, y)

# Get the selected feature rankings and mask of selected features
rfe = pipeline.named_steps['rfe']
selected_features = X.columns[rfe.support_]
selected_features


Index(['person_home_ownership_OWN', 'person_home_ownership_RENT', 'loan_grade',
       'loan_amnt', 'loan_percent_income'],
      dtype='object')

>3.) Using PCA

In [15]:
# Apply PCA to reduce the dimensions
pca = PCA(n_components=2)  # Reduce to 2 components
X_pca = pca.fit_transform(X)

# Get the principal components (linear combination of original features)
print(pca.explained_variance_)
principal_components = pca.components_
principal_components

[5.39185939e+08 1.78007246e+07]


array([[ 7.39941971e-06,  6.19182594e-09, -6.61556571e-07,
        -6.74405496e-06,  7.12634169e-09, -2.38597729e-07,
         1.37832932e-06, -1.39199151e-06,  8.91422643e-08,
         1.55991311e-07,  7.76734134e-07, -7.76734134e-07,
        -4.14348663e-06,  1.77353179e-05,  9.97641560e-01,
         3.05792870e-05,  6.86390293e-02, -1.24081085e-05,
        -1.16291313e-06,  9.87768900e-06],
       [-2.40897069e-06,  5.33015953e-08,  2.25628556e-08,
         2.33310624e-06, -6.07996048e-07, -4.76151057e-07,
         1.37379144e-06, -3.93443394e-07, -1.81569066e-07,
         2.85368125e-07, -2.67995551e-06,  2.67995551e-06,
         2.94223532e-05,  6.87111612e-06, -6.86390285e-02,
         1.58452333e-05,  9.97641556e-01,  8.67338457e-05,
         1.66474993e-05,  8.53951939e-06]])

>4.) Using RandomForest or ExtraTrees -> here Random Forest Classifier

In [16]:
# Create and fit the Extra Trees Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X, y)

# Get feature importances
importances = clf.feature_importances_
importances

array([0.02039369, 0.00051842, 0.01445575, 0.03863797, 0.02183651,
       0.00988845, 0.01512355, 0.02033116, 0.00837791, 0.00813823,
       0.00861023, 0.00803023, 0.13492601, 0.04841996, 0.11460972,
       0.06424287, 0.06964287, 0.15374379, 0.2039783 , 0.03609438])

In [17]:
selectors = []

for i in range (len(X.columns)):
    selectors.append([importances[i],X.columns[i]])
selectors

[[0.020393686231753732, 'person_home_ownership_MORTGAGE'],
 [0.0005184163068958717, 'person_home_ownership_OTHER'],
 [0.014455751341772587, 'person_home_ownership_OWN'],
 [0.03863796729939537, 'person_home_ownership_RENT'],
 [0.02183650667756438, 'loan_intent_DEBTCONSOLIDATION'],
 [0.009888454709273117, 'loan_intent_EDUCATION'],
 [0.01512355426481218, 'loan_intent_HOMEIMPROVEMENT'],
 [0.020331163419051747, 'loan_intent_MEDICAL'],
 [0.008377909865287006, 'loan_intent_PERSONAL'],
 [0.008138233214365681, 'loan_intent_VENTURE'],
 [0.008610233210255263, 'cb_person_default_on_file_N'],
 [0.008030225935342219, 'cb_person_default_on_file_Y'],
 [0.13492600961402518, 'loan_grade'],
 [0.04841995592911101, 'person_age'],
 [0.11460972397531058, 'person_income'],
 [0.06424287021232161, 'person_emp_length'],
 [0.06964287201304846, 'loan_amnt'],
 [0.15374378819162918, 'loan_int_rate'],
 [0.20397829553768043, 'loan_percent_income'],
 [0.036094382051104455, 'cb_person_cred_hist_length']]

In [19]:
# Sort selectors by the first element (importance) in descending order
sorted_selectors = sorted(selectors, key=lambda x: x[0], reverse=True)

# Extract the top 5 feature names
top_5_features = [column for _, column in sorted_selectors[:5]]
top_5_features


['loan_percent_income',
 'loan_int_rate',
 'loan_grade',
 'person_income',
 'loan_amnt']

>5.) Using correlation matrix


In [21]:
correlation_matrix = df.corr()
status_correlation = correlation_matrix['status']

# Sort the correlations in descending order
sorted_correlations = status_correlation.abs().sort_values(ascending=False)

# Select the top 12 features (excluding 'status' itself)
top_features = sorted_correlations[:6]
top_features

status                        1.000000
loan_grade                    0.396437
loan_int_rate                 0.344314
loan_percent_income           0.302362
person_income                 0.209874
person_home_ownership_RENT    0.207639
Name: status, dtype: float64

Finnaly after seeing all results the selected features are `[
    'loan_grade',
    'loan_int_rate',
    'loan_percent_income',
    'person_income',
    'person_home_ownership_RENT',
]`

In [22]:
X = df[[
    'loan_grade',
    'loan_int_rate',
    'loan_percent_income',
    'person_income',
    'person_home_ownership_RENT',
]]

In [23]:
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


**BUILDING A MODEL**

>1.) Random Forest

In [24]:
# Initialize the classifier
model = RandomForestClassifier(random_state=42)

# Train the model
model.fit(X_train, Y_train)


RandomForestClassifier(random_state=42)

In [27]:
# Make predictions on the test set
Y_pred = model.predict(X_test)

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(Y_test, Y_pred))

print("\nClassification Report:")
print(classification_report(Y_test, Y_pred))

# Calculate accuracy
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {accuracy:.2f}")


# Step 1: Get the predicted probabilities for the positive class
Y_prob = model.predict_proba(X_test)[:, 1]  # Probabilities of the positive class (status = 1)

# Step 2: Calculate the ROC AUC score
roc_auc = roc_auc_score(Y_test, Y_prob)
roc_auc*100

Confusion Matrix:
[[8425  241]
 [ 536  775]]

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.97      0.96      8666
           1       0.76      0.59      0.67      1311

    accuracy                           0.92      9977
   macro avg       0.85      0.78      0.81      9977
weighted avg       0.92      0.92      0.92      9977

Accuracy: 0.92


90.08525651418707

>2.) Logistic Regression


In [28]:
# Initialize the classifier with Logistic Regression
model = LogisticRegression(max_iter=500, random_state=42, solver='lbfgs')  # Increase max_iter if needed

# Train the model
model.fit(X_train, Y_train)

LogisticRegression(max_iter=500, random_state=42)

In [29]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score

# Make predictions on the test set
Y_pred = model.predict(X_test)

# Evaluate the model with a confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(Y_test, Y_pred))

print("\nClassification Report:")
print(classification_report(Y_test, Y_pred))

# Calculate accuracy
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Calculate ROC AUC score
# Step 1: Get the predicted probabilities for the positive class (status = 1)
Y_prob = model.predict_proba(X_test)[:, 1]

# Step 2: Calculate the ROC AUC score
roc_auc = roc_auc_score(Y_test, Y_prob)
print(f"ROC AUC Score: {roc_auc:.2f}")


Confusion Matrix:
[[8467  199]
 [ 849  462]]

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      8666
           1       0.70      0.35      0.47      1311

    accuracy                           0.89      9977
   macro avg       0.80      0.66      0.71      9977
weighted avg       0.88      0.89      0.88      9977

Accuracy: 0.89
ROC AUC Score: 0.86


**RANDOM FOREST WITH THIS DATA IS MORE PREFFERED WITH THESE FEATURES AS METRICS > 90% FOR IT -> GOOD MODEL**

***WE CAN USE BUILTIN DATASETS FROM SKLEAN TOO -> DEMONSTRATED BELOW***

In [30]:


# Import necessary libraries
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.datasets import load_iris
import pandas as pd

# Load a dataset (example: Iris dataset)
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)  # Feature matrix
y = pd.Series(data.target)  # Target variable

# Apply SelectKBest using f_classif to select top 2 features
k = 2  # Number of top features to select
selector = SelectKBest(score_func=f_classif, k=k)

# Fit the selector to the data (X: features, y: target)
X_new = selector.fit_transform(X, y)

# Get the selected feature names
selected_features = X.columns[selector.get_support()]

# Display results
print("Selected features:", selected_features)

# Optionally, convert the selected features back into a DataFrame
X_selected = pd.DataFrame(X_new, columns=selected_features)

# Display the transformed data with selected features
selected_features

selector.scores_
selector.pvalues_


Selected features: Index(['petal length (cm)', 'petal width (cm)'], dtype='object')


array([1.66966919e-31, 4.49201713e-17, 2.85677661e-91, 4.16944584e-85])